In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Assuming you have a dataset in a DataFrame named 'data'
# Columns 'X1', 'X2', ..., 'Xn' are your features, and 'target' is your binary target variable.

# Step 1: Prepare your data
data = pd.read_csv("dataset.csv")


# Display the first few rows of the dataset
print(data.head())
data['Churn?'] = np.where(dataset['Churn?'] == 'True.',1, 0)
data['Int\'l Plan'] = np.where(dataset['Int\'l Plan'] == 'yes',1, 0)
data['VMail Plan'] = np.where(dataset['VMail Plan'] == 'yes',1, 0)
# Step 1: Prepare your data
X = data.drop(['State', 'Phone', 'Churn?'], axis=1)  # Drop non-numeric or non-binary columns
y = data['Churn?'].map(lambda x: 1 if x == True else 0)  # Convert 'True' to 1 and 'False' to 0

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 4: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 5: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Output the results
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)


  State  Account Length  Area Code     Phone Int'l Plan VMail Plan  \
0    KS             128        415  382-4657         no        yes   
1    OH             107        415  371-7191         no        yes   
2    NJ             137        415  358-1921         no         no   
3    OH              84        408  375-9999        yes         no   
4    OK              75        415  330-6626        yes         no   

   VMail Message  Day Mins  Day Calls  Day Charge  ...  Eve Calls  Eve Charge  \
0             25     265.1        110       45.07  ...         99       16.78   
1             26     161.6        123       27.47  ...        103       16.62   
2              0     243.4        114       41.38  ...        110       10.30   
3              0     299.4         71       50.90  ...         88        5.26   
4              0     166.7        113       28.34  ...        122       12.61   

   Night Mins  Night Calls  Night Charge  Intl Mins  Intl Calls  Intl Charge  \
0       244.

ValueError: could not convert string to float: 'no'